In [1]:
import pandas as pd
import os

**In this script, it was aimed to obtain (load and clean) the Table 1 data (Causes of death – diseases of the circulatory system, residents, 2022) in a way to make it possible to process afterwards. The below steps have been followed here:**
* Step 1: Load the Excel file and inspect sheet names
* Step 2: Load 'Table 1' with multi-level headers
* Step 3: Flatten the header (remove 'Unnamed' parts)
* Step 4: Drop empty first two columns
* Step 5: Rename columns clearly
* Step 6: Clean up and filter country rows
* Step 7: Convert numeric columns
* Step 8: Remove footnotes and fix footnote marker like " (¹)" at the country names (e.g. "Türkiye (¹)" -> "Türkiye").
* Step 9: Save cleaned file 

**Notes on Table 1:** CVD stands for cardiovascular disease, STD stands for standardised death rate. Please refer to data description in the readme file for details. 



In [2]:
xlsx = pd.ExcelFile("Cardiovascular_diseases_Health2025.xlsx")
print("Sheet names:", xlsx.sheet_names)

df_raw = pd.read_excel(xlsx, sheet_name="Table 1", header=[0, 1, 2])

def flatten_col(col_tuple):
    parts = [str(x) for x in col_tuple if ('Unnamed' not in str(x)) and (str(x) != 'nan')]
    return "_".join(parts).strip()

df_raw.columns = [flatten_col(col) for col in df_raw.columns]
print("Flattened columns:", df_raw.columns.tolist())

df_raw = df_raw.drop(df_raw.columns[:2], axis=1)
print("Columns after dropping first two:", df_raw.columns.tolist())

df_raw.columns = [
    "Country",
    "Number_of_deaths",
    "Share_of_all_deaths_total",
    "Share_male",
    "Share_female",
    "SDR_total",
    "SDR_male",
    "SDR_female",
    "SDR_under_65",
    "SDR_65plus"
]

df_raw = df_raw.dropna(subset=["Country"]).reset_index(drop=True)
df_raw["Country"] = df_raw["Country"].str.strip()

df_raw = df_raw[
    ~df_raw["Country"].str.contains(
        "Cardiovascular diseases|Table 1|Source|For text|Definition|Bookmark",
        case=False,
        na=False
    )
]

cols_to_convert = df_raw.columns.drop("Country")
for col in cols_to_convert:
    df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

df_raw["Country"] = df_raw["Country"].str.replace(r"\s*\(.*\)", "", regex=True).str.strip()

output_dir = "data/processed"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "deaths_clean.csv")
df_raw.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")

print(df_raw.info())
print(df_raw.head())

Sheet names: ['BulkList', 'Table 1', 'Table 2', 'Table 3']
Flattened columns: ['s', 's', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures']
Columns after dropping first two: ['s_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in the European Union – facts and figures', 's_Health in